# Mastering with ROS: Turtlebot3

<img src="img/robotis_logo.png" width="400" />

<img src="img/t3_navigation.png" width="600"/>

<img src="img/robotignite_logo_text.png" width="400"/>

## Unit 2: Navigation with Turtlebot3

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: **4h**

The first thing you will need for a robot that patroles is the ability to move around without crashing into everything, right?. Well, that's what you are going to learn in this Unit! For this you will learn:

* How to create a map of an environment
* Localize the robot within the map
* Path Planning with Obstacle Avoidance
* Send a sequence of waypoints and execute those movements

<p style="background:green;color:white;">END OF SUMMARY</p>

## Creating a Map

The first thing you need in order to perform ROS Navigation, is to create a Map of the environment you want to navigate. For that, you are going to need the **<a href="http://wiki.ros.org/gmapping">slam_gmapping</a>** node that the Navigation Stack provides. For seeing how to do this, follow the next exercise:

<p style="background:#EE9023;color:white;">Exercise 2.1</p>

a) Firs of all, let's create a new package where we'll put all the files related to navigation.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
catkin_create_pkg t3_navigation

b) Inside this new package, let's create 2 new directories: one named **launch** and the other one named **param**.

c) Now, let's create a launch file in order to start our slam_gmapping node!

**start_mapping.launch**

In [ ]:
<launch>
  <!-- Turtlebot3 -->
  <include file="$(find turtlebot3_bringup)/launch/turtlebot3_remote.launch" />

  <!-- Gmapping -->
  <node pkg="gmapping" type="slam_gmapping" name="turtlebot3_slam_gmapping" output="screen">
    <param name="base_frame" value="base_footprint"/>
    <param name="odom_frame" value="odom"/>
    <param name="map_update_interval" value="2.0"/>
    <param name="maxUrange" value="4.0"/>
    <param name="minimumScore" value="100"/>
    <param name="linearUpdate" value="0.2"/>
    <param name="angularUpdate" value="0.2"/>
    <param name="temporalUpdate" value="0.5"/>
    <param name="delta" value="0.05"/>
    <param name="lskip" value="0"/>
    <param name="particles" value="120"/>
    <param name="sigma" value="0.05"/>
    <param name="kernelSize" value="1"/>
    <param name="lstep" value="0.05"/>
    <param name="astep" value="0.05"/>
    <param name="iterations" value="5"/>
    <param name="lsigma" value="0.075"/>
    <param name="ogain" value="3.0"/>
    <param name="srr" value="0.01"/>
    <param name="srt" value="0.02"/>
    <param name="str" value="0.01"/>
    <param name="stt" value="0.02"/>
    <param name="resampleThreshold" value="0.5"/>
    <param name="xmin" value="-10.0"/>
    <param name="ymin" value="-10.0"/>
    <param name="xmax" value="10.0"/>
    <param name="ymax" value="10.0"/>
    <param name="llsamplerange" value="0.01"/>
    <param name="llsamplestep" value="0.01"/>
    <param name="lasamplerange" value="0.005"/>
    <param name="lasamplestep" value="0.005"/>

    </node>
</launch>

The most important parameters in these file are:

* **maxUrange**: This parameter sets how far your laser will be considered to create the map. Greater range will create maps faster and its less probable that the robot gets lost. The downside its that consumes more resources.
* **throttle_scans**: Very usefull to lower resource consumption.

If you need more information about all these parameters, please go to the slam_gmapping node docs: http://wiki.ros.org/gmapping

d) Now, you can proceed to start this launch file. 

e) Let's now launch RViz in order to be able to visualize the mapping process. For that, you have 2 options:

* You can launch a naked Rviz and add the proper elements for visualizing the mapping process (**LaserScan** and **Map**)
* You can use the following command, which launches a predefined configuration of RVIZ for Mapping:

In [ ]:
rosrun rviz rviz -d `rospack find turtlebot3_slam`/rviz/turtlebot3_slam.rviz

You should see something like this:

<img src="img/initial_map.png" width="600" />

f) Now, you can start moving the robot around the environment in order to generate a full map. At the end, you should end up with something like this:

<img src="img/full_map.png" width="600" />

g) You can also play with the values in the **maxUrange** parameter, to see how it affects the mapping process.

<p style="color:red;">**IMPORTANT**: DO NOT CLOSE anything when you finalise with the exercise (have created the full map). You will have to work with this.</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.1</p>

Great! So now, you have already created a full map of the environment. So now what? Well, now it's time to save this map, so you can use it in the Pat Planning system!

## Saving the map

Another of the packages available in the ROS Navigation Stack is the map_server package. This package provides the map_saver node, which allows us to access the map data from a ROS Service, and save it into a file.

You can save the built map at anytime by using the following command:

In [ ]:
rosrun map_server map_saver -f name_of_map

This command will get the map data from the map topic, and write it out into 2 files, name_of_map.pgm and name_of_map.yaml.

<p style="background:#EE9023;color:white;">**Exercise 2.2**</p>

a) Save the map created in the previous exercise into a file.
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #4</p>
</th>
</tr>
</table>
<br>

In [ ]:
roscd t3_navigation;
mkdir maps;
cd maps;
rosrun map_server map_saver -f my_map;

<p style="background:#EE9023;color:white;">**End of Exercise 2.2**</p>

You should end up with 2 new files: **my_map.yaml** and **my_map.pgm**.

The PGM file is the one that contains the occupancy data of the map (the really important data), and the YAML file contains some metadata about the map, like the map dimensions and resolution, or the path to the PGM file.

**my_map.yaml**

image: my_map.pgm
resolution: 0.050000
origin: [-10.000000, -10.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196

**my_map.pgm**

<img src="img/map_file.png" width="500" />

## Localize the robot

So after generating the map, the next thing we'll need to do is to be able to localize the robot into that map. If we don't do this, the map would be totally useless. Right?

For that, we are going to use the **amcl** node from the Navigation Stack. So, as you did for the mapping process, let's create a launch file in order to start this node.

<p style="background:#EE9023;color:white;">Exercise 2.3</p>

a) Inside your package, create a new launch file in order to start the localization node.

**start_localization.launch**

In [ ]:
<launch>

  <include file="$(find turtlebot3_bringup)/launch/turtlebot3_remote.launch" />
    
  <arg name="map_file" default="$(find t3_navigation)/maps/my_map.yaml"/>
  <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)" />
    
  <arg name="scan_topic"     default="scan"/>
  <arg name="initial_pose_x" default="0.0"/>
  <arg name="initial_pose_y" default="0.0"/>
  <arg name="initial_pose_a" default="0.0"/>

  <node pkg="amcl" type="amcl" name="amcl">

    <param name="min_particles"             value="500"/>
    <param name="max_particles"             value="3000"/>
    <param name="kld_err"                   value="0.02"/>
    <param name="update_min_d"              value="0.20"/>
    <param name="update_min_a"              value="0.20"/>
    <param name="resample_interval"         value="1"/>
    <param name="transform_tolerance"       value="0.5"/>
    <param name="recovery_alpha_slow"       value="0.00"/>
    <param name="recovery_alpha_fast"       value="0.00"/>
    <param name="initial_pose_x"            value="$(arg initial_pose_x)"/>
    <param name="initial_pose_y"            value="$(arg initial_pose_y)"/>
    <param name="initial_pose_a"            value="$(arg initial_pose_a)"/>
    <param name="gui_publish_rate"          value="50.0"/>

    <remap from="scan"                      to="$(arg scan_topic)"/>
    <param name="laser_max_range"           value="3.5"/>
    <param name="laser_max_beams"           value="180"/>
    <param name="laser_z_hit"               value="0.5"/>
    <param name="laser_z_short"             value="0.05"/>
    <param name="laser_z_max"               value="0.05"/>
    <param name="laser_z_rand"              value="0.5"/>
    <param name="laser_sigma_hit"           value="0.2"/>
    <param name="laser_lambda_short"        value="0.1"/>
    <param name="laser_likelihood_max_dist" value="2.0"/>
    <param name="laser_model_type"          value="likelihood_field"/>

    <param name="odom_model_type"           value="diff"/>
    <param name="odom_alpha1"               value="0.1"/>
    <param name="odom_alpha2"               value="0.1"/>
    <param name="odom_alpha3"               value="0.1"/>
    <param name="odom_alpha4"               value="0.1"/>
    <param name="odom_frame_id"             value="odom"/>
    <param name="base_frame_id"             value="base_footprint"/>

  </node>
</launch>

The most important parameters in these file are:

* **min_particles, max_particles**: This parameter sets the number of particles that the filter will use in order to localize the robot. The more you use, the more precise the localization will be, but the more resources it will consume.
* **laser_max_range**: Max range of the laser beams.

If you need more information about all these parameters, please go to the amcl node docs: http://wiki.ros.org/amcl

d) Now, you can proceed to start this launch file. 

e) Let's now launch RViz in order to be able to visualize the localization process. You can use the same setup you used for the mapping process, adding 1 more display: **Pose Array**.

You should see something like this:

<img src="img/localization_start.png" width="600" />

f) Now, you can start moving the robot around the environment in order to localize the robot. As you move the robot, you will se in RViz how the particles keep getting closer, which means that the estimated poses of the robot are getting closer to the real place. This is a test on how good your localization system is working.

<img src="img/localization_end.png" width="600" />

g) You can also play with the values in the **min_particles** and **max_particles** parameters, to see how it affects the localization process.

<p style="background:#EE9023;color:white;">End of Exercise 2.3</p>

Great! So at this point, you've already built a map of the environment and you are able to localize the Turtlebot3 robot on the map. This means eberything is ready to Navigate the robot!

## Path Planning and Obstacle Avoidance

For doing Path Planning, you'll need to combine everything you've donetill now. Plus, you will have to use the **move_base** node from the Navigation Stack, which will manage all the Path Planning system for you. So, as you've done in the previous exercises, let's create our launch file in order to launch the Path Planning system. This time, though, you'll have some extra work to do, since there are a lot of parameters involved that you'll need to set. But don't worry, you can follow the next exercise in order to be guided through the process!

<p style="background:#EE9023;color:white;">Exercise 2.4</p>

a) Inside your package, create a new launch file in order to start the move_base node.

**start_navigation.launch**

In [ ]:
<launch>
  <arg name="model" default="burger" doc="model type [burger, waffle]"/>

  <!-- Turtlebot3 -->
  <include file="$(find turtlebot3_bringup)/launch/turtlebot3_remote.launch" />

  <!-- Map server -->
  <arg name="map_file" default="$(find t3_navigation)/maps/my_map.yaml"/>
  <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)">
  </node>

  <!-- AMCL -->
  <include file="$(find t3_navigation)/launch/start_localization.launch"/>

  <!-- move_base -->
  <arg name="cmd_vel_topic" default="/cmd_vel" />
  <arg name="odom_topic" default="odom" />
  <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">
    <param name="base_local_planner" value="dwa_local_planner/DWAPlannerROS" />

    <rosparam file="$(find t3_navigation)/param/costmap_common_params_$(arg model).yaml" command="load" ns="global_costmap" />
    <rosparam file="$(find t3_navigation)/param/costmap_common_params_$(arg model).yaml" command="load" ns="local_costmap" />
    <rosparam file="$(find t3_navigation)/param/local_costmap_params.yaml" command="load" />
    <rosparam file="$(find t3_navigation)/param/global_costmap_params.yaml" command="load" />
    <rosparam file="$(find t3_navigation)/param/move_base_params.yaml" command="load" />
    <rosparam file="$(find t3_navigation)/param/dwa_local_planner_params.yaml" command="load" />

    <remap from="cmd_vel" to="$(arg cmd_vel_topic)"/>
    <remap from="odom" to="$(arg odom_topic)"/>
  </node>
</launch>

We can see there three main parts:
* **Map Server**: The node that publishes the map saved.
* **AMCL**: The node that localises the robot based on laser readings
* **move_base**: The node that will provide Path Planning and Obstacle Avoidance

So, you will just need to create all this parameter files that are being loaded for the move_base node.

b) Create all the parameter files required by the move_base node.

**costmap_common_params_burger.yaml**

In [ ]:
obstacle_range: 2.5
raytrace_range: 3.5
footprint: [[-0.110, -0.090], [-0.110, 0.090], [0.041, 0.090], [0.041, -0.090]]
#robot_radius: 0.105
inflation_radius: 0.15
cost_scaling_factor: 0.5
map_type: costmap
transform_tolerance: 0.2
observation_sources: scan
scan: {data_type: LaserScan, topic: scan, marking: true, clearing: true}

Just note the observation_sources parameters, that uses the topic /scan to read the laser readings. The camera is not used for navigation.
Also note the ray trace range is only 3.5 meters. This is just to make detections faster and not have in mind areas that aren't close enough.

**local_costmap_params.yaml**

In [ ]:
local_costmap:
  global_frame: /odom
  robot_base_frame: /base_footprint
  update_frequency: 2.0
  publish_frequency: 0.5
  static_map: false
  rolling_window: true
  width: 3.5
  height: 3.5
  resolution: 0.05
  transform_tolerance: 1.0

Note that the static_map parmeter is set to False. This is because the local costmap is built from the laser readings, not from any static map.

**global_costmap_params.yaml**

In [ ]:
global_costmap:
  global_frame: /map
  robot_base_frame: /base_footprint
  update_frequency: 2.0
  publish_frequency: 0.1
  static_map: true
  transform_tolerance: 1.0

Note that the static_map parmeter is here set to True. This is because the global costmap is built from the static map you created in previous steps.

**move_base_params.yaml**

In [ ]:
shutdown_costmaps: false
controller_frequency: 3.0
controller_patience: 1.0
planner_frequency: 2.0
planner_patience: 1.0
oscillation_timeout: 10.0
oscillation_distance: 0.2
conservative_reset_dist: 0.10

These are some general parameters for the move_base node.

**dwa_local_planner_params.yaml**

In [ ]:
DWAPlannerROS:

# Robot Configuration Parameters
  max_vel_x: 0.18
  min_vel_x: -0.18

  max_vel_y: 0.0
  min_vel_y: 0.0

  # The velocity when robot is moving in a straight line
  max_trans_vel:  0.18
  min_trans_vel:  0.05
#  trans_stopped_vel: 0.01

  max_rot_vel: 1.8
  min_rot_vel: 0.7
#  rot_stopped_vel: 0.01

  acc_lim_x: 2.0
  acc_lim_theta: 2.0
  acc_lim_y: 0.0

# Goal Tolerance Parametes
  yaw_goal_tolerance: 0.15
  xy_goal_tolerance: 0.05

# Forward Simulation Parameters
  sim_time: 3.5
  vx_samples: 20
  vy_samples: 0
  vtheta_samples: 40

# Trajectory Scoring Parameters
  path_distance_bias: 32.0
  goal_distance_bias: 24.0
  occdist_scale: 0.04
  forward_point_distance: 0.325
  stop_time_buffer: 0.2
  scaling_speed: 0.25
  max_scaling_factor: 0.2

# Oscillation Prevention Parameters
  oscillation_reset_dist: 0.05

# Debugging
  publish_traj_pc : true
  publish_cost_grid_pc: true
 # global_frame_id: odom

These are some parameters related to the local planner.

<p style="color: red;">**NOTE**:If you want more details on how all these parameter files work, you can have a look at the **ROS Navigation in 5 Days Course**.</p>

c) Execute your launch file in order to start the navigation system.

In [ ]:
roslaunch t3_navigation start_navigation.launch

d) Let's now launch RViz in order to be able to visualize the Path Planning process. For that, you have 2 options:

* You can launch a naked Rviz and add the proper elements for visualizing the Navigation process
* You can use the following command, which launches a predefined configuration of RVIZ for Navigation:

In [ ]:
rosrun rviz rviz -d `rospack find turtlebot3_navigation`/rviz/turtlebot3_nav.rviz

You should see something like this:

<img src="img/demo1.png" width="600" />

e) Use the 2D Pose Estimate tool in Rviz to Localize the Robot in the Map. Just as you did in the Demo of the previous Unit.

f) Use the 2D Nav Goal tool in Rviz to send a Goal(desired pose) to the Robot. Just as you did in the Demo of the previous Unit.

You should now see the Turtlebot3 robot going to that position in the simulation. In Rviz, you can also visualize the planned path that it follows.

<img src="img/demo3.png" width="600" />

g) You can also play with some of the values in the parameter files you created previously, to see how they affect the Path Planning process.

<p style="background:#EE9023;color:white;">End of Exercise 2.4</p>

## Add WayPoints

So there you have it. Now your Turtlebot3 robot can navigate around one position at a time. But what if you want to generate in an easy way a sequence of waypoints so that it patroles around the **main interest points of the area**?

You are going to use a ROS package called **follow_waypoints**, you can get more information http://wiki.ros.org/follow_waypoints.
This package basically tracks the **Estimate pose** that you place in RVIZ and stores it. Then when you have finished you just have to publish in a topic that starts sending those positions to the movebase system.

### Steps to do it:

The first step is to start the **start_navigation_with_map.launch** if you don't have it running:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch t3_navigation start_navigation.launch

The Second step is to start the waypoint server. This server will listen to publications into the topic **/initialpose** and store those poses until its instructed to send them to **move_base** to be executed:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
roslaunch follow_waypoints follow_waypoints.launch

Then you have to start RVIZ, and add a **PoseArray** element, susbcribed to the topic **/waypoints**, and change its name to WayPoints. This will now show all the waypoints you set from now on.

You can change the colour also if you want. In this example its changed to **green** to see it better and differenciate from the posearray of **psoition estimates**.

Now you have to set the waypoints. For this you have to select the **PoseEstimate** and set it where you want it. Because this will move the estimated pose of the robot around its important that **the last waypoint is where the robot is right now**. Otherwise it will be harder for the robot to get to the place. 

<img src="img/create_waypoints.gif" width="600"></img>

You should get something similar to this:

<img src="img/all_waypoints.png" width="600"></img>

You should also get in webshell where you launched the waypoint server a message stating that it recieved the waypoint:

In [ ]:
[INFO][139816504842048][/follow_waypoints/execute:92]: Waiting to recieve waypoints via Pose msg on topic /initialpose         
[INFO][139816504842048][/follow_waypoints/execute:93]: To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -
1'                                                                                                                             
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                       

Now you just have to publish in the topic **/path_ready** to start sending waypoints to the movebase.

If you want to see the global path generated just activate it in RVIZ.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p><br>
rostopic pub /path_ready std_msgs/Empty -1
</th>
</tr>
</table>

You should get something similar to this for each waypoint executed. Bare in mind that because you move the robot around, it will probably first do a recovery behaviour to clean up all the false obstacles generated through changing the estimate pose:

<img src="img/follow_waypoints.gif" width="600"></img>

As you can see, its a nice way to do it , but its a bit cumbersome and messy. Specially because it has to change the initial position of the robot which is not very desirable. This is because this waypoint manager listenes to the topic **/initialpose**. Th eproblem is that **amlc localiser** also. Thats why when changed, the robot localises itself also in another place.<br>
So the next step is creating your own version. So you will part from the code that you just used that is installed in the system and is courtsesy of <a href="https://github.com/danielsnider">Daniel Snider</a> ,who created the original Waypoints manager <a href="https://github.com/danielsnider/follow_waypoints">Program</a>.

In order to avoid touching the **/initialpose**, the topic that the **follow_points** listens to, this has to be changed. Not only that put it has to be configurable. See here the final code:

**custom_follow_waypoints.py**

In [ ]:
#!/usr/bin/env python

import threading
import rospy
import actionlib

from smach import State,StateMachine
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import PoseWithCovarianceStamped, PoseArray
from std_msgs.msg import Empty

waypoints = []

class FollowPath(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'], input_keys=['waypoints'])
        self.frame_id = rospy.get_param('~goal_frame_id','map')
        # Get a move_base action client
        self.client = actionlib.SimpleActionClient('move_base', MoveBaseAction)
        rospy.loginfo('Connecting to move_base...')
        self.client.wait_for_server()
        rospy.loginfo('Connected to move_base.')

    def execute(self, userdata):
        global waypoints
        # Execute waypoints each in sequence
        for waypoint in waypoints:
            # Break if preempted
            if waypoints == []:
                rospy.loginfo('The waypoint queue has been reset.')
                break
            # Otherwise publish next waypoint as goal
            goal = MoveBaseGoal()
            goal.target_pose.header.frame_id = self.frame_id
            goal.target_pose.pose.position = waypoint.pose.pose.position
            goal.target_pose.pose.orientation = waypoint.pose.pose.orientation
            rospy.loginfo('Executing move_base goal to position (x,y): %s, %s' %
                    (waypoint.pose.pose.position.x, waypoint.pose.pose.position.y))
            rospy.loginfo("To cancel the goal: 'rostopic pub -1 /move_base/cancel actionlib_msgs/GoalID -- {}'")
            self.client.send_goal(goal)
            self.client.wait_for_result()
        return 'success'

def convert_PoseWithCovArray_to_PoseArray(waypoints):
    """Used to publish waypoints as pose array so that you can see them in rviz, etc."""
    poses = PoseArray()
    poses.header.frame_id = 'map'
    poses.poses = [pose.pose.pose for pose in waypoints]
    return poses

class GetPath(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'], input_keys=['waypoints'], output_keys=['waypoints'])
        # Create publsher to publish waypoints as pose array so that you can see them in rviz, etc.
        self.poseArray_publisher = rospy.Publisher('/waypoints', PoseArray, queue_size=1)
        self._custom_waypointstopic = rospy.get_param('~custom_waypointstopic','/my_waypoints_list')
        # Start thread to listen for reset messages to clear the waypoint queue
        def wait_for_path_reset():
            """thread worker function"""
            global waypoints
            while not rospy.is_shutdown():
                data = rospy.wait_for_message('/path_reset', Empty)
                rospy.loginfo('Recieved path RESET message')
                self.initialize_path_queue()
                rospy.sleep(3) # Wait 3 seconds because `rostopic echo` latches
                               # for three seconds and wait_for_message() in a
                               # loop will see it again.
        reset_thread = threading.Thread(target=wait_for_path_reset)
        reset_thread.start()

    def initialize_path_queue(self):
        global waypoints
        waypoints = [] # the waypoint queue
        # publish empty waypoint queue as pose array so that you can see them the change in rviz, etc.
        self.poseArray_publisher.publish(convert_PoseWithCovArray_to_PoseArray(waypoints))

    def execute(self, userdata):
        global waypoints
        self.initialize_path_queue()
        self.path_ready = False

        # Start thread to listen for when the path is ready (this function will end then)
        def wait_for_path_ready():
            """thread worker function"""
            data = rospy.wait_for_message('/path_ready', Empty)
            rospy.loginfo('Recieved path READY message')
            self.path_ready = True
        ready_thread = threading.Thread(target=wait_for_path_ready)
        ready_thread.start()

        topic = self._custom_waypointstopic
        rospy.loginfo("Waiting to recieve waypoints via Pose msg on topic %s" % topic)
        rospy.loginfo("To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -1'")

        # Wait for published waypoints
        while not self.path_ready:
            try:
                pose = rospy.wait_for_message(topic, PoseWithCovarianceStamped, timeout=1)
            except rospy.ROSException as e:
                if 'timeout exceeded' in e.message:
                    continue  # no new waypoint within timeout, looping...
                else:
                    raise e
            rospy.loginfo("Recieved new waypoint")
            waypoints.append(pose)
            # publish waypoint queue as pose array so that you can see them in rviz, etc.
            self.poseArray_publisher.publish(convert_PoseWithCovArray_to_PoseArray(waypoints))

        # Path is ready! return success and move on to the next state (FOLLOW_PATH)
        return 'success'

class PathComplete(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'])

    def execute(self, userdata):
        rospy.loginfo('###############################')
        rospy.loginfo('##### REACHED FINISH GATE #####')
        rospy.loginfo('###############################')
        return 'success'

def main():
    rospy.init_node('custom_follow_waypoints')

    sm = StateMachine(outcomes=['success'])

    with sm:
        StateMachine.add('GET_PATH', GetPath(),
                           transitions={'success':'FOLLOW_PATH'},
                           remapping={'waypoints':'waypoints'})
        StateMachine.add('FOLLOW_PATH', FollowPath(),
                           transitions={'success':'PATH_COMPLETE'},
                           remapping={'waypoints':'waypoints'})
        StateMachine.add('PATH_COMPLETE', PathComplete(),
                           transitions={'success':'GET_PATH'})

    outcome = sm.execute()

if __name__ == "__main__":
    main()

The new topic is retrived through a parameter setting.

In [ ]:
self._custom_waypointstopic = rospy.get_param('~custom_waypointstopic','/my_waypoints_list')

This means that when this node is launched, it will have to be set this parameter if we don't want it to have its default value **/my_waypoints_list**.<br>
Here you have an example of how it should be launched:

**start_follow_waypoints.launch**

In [ ]:
<launch>
    <env name="ROSCONSOLE_FORMAT" value="[${severity}][${thread}][${node}/${function}:${line}]: ${message}"/>
     <arg name="waypoints_topic" default="/my_t3_waypoints"/>
    
    <node pkg="t3_navigation" type="custom_follow_waypoints.py" name="custom_follow_waypoints" output="screen" clear_params="true">
        <param name="goal_frame_id" value="map"/>
        <param name="custom_waypointstopic" value="$(arg waypoints_topic)"/>
    </node>

</launch>

Lets comment on various elements here:

In [ ]:
<env name="ROSCONSOLE_FORMAT" value="[${severity}][${thread}][${node}/${function}:${line}]: ${message}"/>

This line changes the way the logs are shown, in this case it will show which node, function and line in the code is that log executed.

In [ ]:
<arg name="waypoints_topic" default="/my_t3_waypoints"/>

Here you state which topic where you will have to publish the waypoint.

In [ ]:
<param name="goal_frame_id" value="map"/>
<param name="custom_waypointstopic" value="$(arg waypoints_topic)"/>

Here you state which is the frame and the topic params values.

<p style="background:#EE9023;color:white;">Exercise 2.5</p>

Now create a python script that generates those waypoints and when finished executes the moving system to follow those waypoints.<br>
To do so, you will need to publish directly into the topic **/my_t3_waypoints** or the value that you stated in tha launch file **start_follow_waypoints.launch**.<br>
Then you will have to publish to start in the topic **/path_ready**.<br>
Tasks:

* Make it generate circular paths
* Eight shaped paths

<p style="background:#EE9023;color:white;">Exercise 2.5</p>

<p style="background:green;color:white;">Solution Exercise 2.5</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png" width="600" />

Here you have an example of a publisher that generates a straight line waypoint path and executes it.

**waypoint_publisher.py**

In [ ]:
#!/usr/bin/env python
# license removed for brevity
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import PoseWithCovarianceStamped
import time
import tf
from std_msgs.msg import Empty

"""
user:~$ rosmsg show geometry_msgs/PoseWithCovarianceStamped                                                                                                               
std_msgs/Header header                                                                                                                                                    
  uint32 seq                                                                                                                                                              
  time stamp                                                                                                                                                              
  string frame_id                                                                                                                                                         
geometry_msgs/PoseWithCovariance pose                                                                                                                                     
  geometry_msgs/Pose pose                                                                                                                                                 
    geometry_msgs/Point position                                                                                                                                          
      float64 x                                                                                                                                                           
      float64 y                                                                                                                                                           
      float64 z                                                                                                                                                           
    geometry_msgs/Quaternion orientation                                                                                                                                  
      float64 x                                                                                                                                                           
      float64 y                                                                                                                                                           
      float64 z                                                                                                                                                           
      float64 w                                                                                                                                                           
  float64[36] covariance 
"""

def talker():
    pub_wp = rospy.Publisher('/my_t3_waypoints', PoseWithCovarianceStamped, queue_size=1)
    
    pub_init_wp = rospy.Publisher('/path_ready', Empty, queue_size=1)
    
    rospy.init_node('waypoint_publisher', anonymous=True)
    rate = rospy.Rate(10) # 10hz
    
    my_wp = PoseWithCovarianceStamped()
    my_wp.header.stamp = rospy.Time.now()
    my_wp.header.frame_id = "/map"
    
    
    init_value = 2.0
    
    roll = 0
    pitch = 0
    yaw = 0.7
    quaternion = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    #type(pose) = geometry_msgs.msg.Pose
    my_wp.pose.pose.orientation.x = quaternion[0]
    my_wp.pose.pose.orientation.y = quaternion[1]
    my_wp.pose.pose.orientation.z = quaternion[2]
    my_wp.pose.pose.orientation.w = quaternion[3]


    for i in range(5):
        rospy.loginfo(str(i))
        my_wp.pose.pose.position.x = float(i) + init_value 
        my_wp.pose.pose.position.y = float(i) + init_value
        
        while not rospy.is_shutdown():
            connections = pub_wp.get_num_connections()
            if connections > 0:
                pub_wp.publish(my_wp)
                break
            rospy.loginfo("Waiting for /my_t3_waypoints topic")
            rate.sleep()
        rospy.loginfo("Published waypoint number ="+str(i))
        time.sleep(2)
    
    start_command = Empty()
    
    while not rospy.is_shutdown():
            connections = pub_init_wp.get_num_connections()
            if connections > 0:
                pub_init_wp.publish(start_command)
                rospy.loginfo("Sent waypoint list execution command")
                break
            rospy.loginfo("Waiting for /path_ready topic")
            rate.sleep()


if __name__ == '__main__':
    try:
        talker()
    except rospy.ROSInterruptException:
        pass

## Congratulations!! You are now capable of creating your own navigation program for indoor with your Turtlebot3 robot.